# Imports

In [ ]:
import os
import gc
import itertools
from google.colab import drive
import math
import requests
import json

import pandas as pd
import numpy as np
import pytz
import datetime as dt
from datetime import datetime, timedelta, timezone
from dateutil import tz
from os.path import isfile, join

# Global Variables and Google Drive Connect

In [ ]:
USER = 'Tom'
TESTING = False

In [ ]:
# Mount drive where you will do your work.
drive.mount('/content/drive')
if USER == 'Tom': 
  root_dir = "/content/drive/My Drive/W210/"
  project_folder = "Capstone/"
elif USER == 'Bri':
  pass
  root_dir = "/content/drive/My Drive/W210/"
  project_folder = "Capstone/Data/"
elif USER == 'Bronte':
  pass
  # root_dir = "/content/drive/My Drive/266/"
  # project_folder = "Final Project/"
elif USER == 'Kurt':
  pass
  root_dir = "/content/drive/My Drive/210/"
  project_folder = "Data/"
else:
  raise Exception("User unrecognized, must connect to shared drive")

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  # to test if all is working, you can uncomment these two lines below--it should write a file to the shared drive
  # !touch 'new_file_in_working_directory.txt'
# print('\nYour working directory was changed to ' + root_dir + project_folder + \
#         "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

os.chdir(root_dir + project_folder)

Mounted at /content/drive


# Optional - Load, Process, and Save Raw Data from Scratch

## EIA Generation and Demand Forecast

In [ ]:
# Read csv files
df = pd.read_excel(f'Data/EIA/Region_Files/Region_CAL.xlsx', sheet_name='Published Hourly Data', parse_dates=True)

In [ ]:
# Replace whitespace in column names with underscore
df.columns = df.columns.str.replace(" ", "_")

# Change to column names to lowercase
df.columns = df.columns.str.lower()

# Remove characters from column names
df.columns = df.columns.str.replace("[(:)]", "")

# Convert str date columns to datetime format
df["utc_time_with_offset"] = pd.to_datetime(df["utc_time"]).dt.tz_localize("UTC")

# set utc_time with UTC timezone object and make index
df.set_index('utc_time_with_offset', inplace=True)
df.index = df.index.tz_convert(timezone.utc)

# drop redundant or unused columns
df = df.drop(columns=['region', 'local_date', 'hour', 'time_zone', 'car', 'cent', 'fla', 'mida',
       'midw', 'ne', 'ny', 'se','ten', 'tex', 'can'])

# # rename all columns by appending "_eia" to their names so one can tell source of data
# df = df.rename(columns={col: col+'_eia' for col in df.columns})

<ipython-input-5-b53ba32ae82d>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace("[(:)]", "")


In [ ]:
df.columns

Index(['utc_time', 'local_time', 'df', 'd', 'ng', 'ti', 'sum_ng', 'ng_col',
       'ng_ng', 'ng_nuc', 'ng_oil', 'ng_wat', 'ng_sun', 'ng_wnd', 'ng_oth',
       'ng_unk', 'sum_trade', 'sum_imports', 'sum_exports', 'cal', 'nw', 'sw',
       'mex', 'balance_ng_d_ti', 'balance_ti_trade', 'balance_ng',
       'co2_factor_col', 'co2_factor_ng', 'co2_factor_oil',
       'co2_emissions_col', 'co2_emissions_ng', 'co2_emissions_oil',
       'co2_emissions_other', 'co2_emissions_generated',
       'co2_emissions_imported', 'co2_emissions_exported',
       'co2_emissions_consumed', 'positive_generation', 'consumed_electricity',
       'co2_emissions_intensity_for_generated_electricity',
       'co2_emissions_intensity_for_consumed_electricity'],
      dtype='object')

In [ ]:
# Add time features
df['local_year'] = df['local_time'].dt.year
df['local_month'] = df['local_time'].dt.month
df['local_week_of_year'] = df['local_time'].dt.isocalendar().week
df['local_day'] = df['local_time'].dt.day
df['local_day_of_week'] = df['local_time'].dt.dayofweek
df['local_day_of_year'] = df['local_time'].dt.dayofyear
df['local_is_weekend'] = df['local_day_of_week'] > 4
df['local_hour'] = df['local_time'].dt.hour
df['local_minute'] = df['local_time'].dt.minute

df['utc_year'] = df['utc_time'].dt.year
df['utc_month'] = df['utc_time'].dt.month
df['utc_week_of_year'] = df['utc_time'].dt.isocalendar().week
df['utc_day'] = df['utc_time'].dt.day
df['utc_day_of_week'] = df['utc_time'].dt.dayofweek
df['utc_day_of_year'] = df['utc_time'].dt.dayofyear
df['utc_is_weekend'] = df['utc_day_of_week'] > 4
df['utc_hour'] = df['utc_time'].dt.hour
df['utc_minute'] = df['utc_time'].dt.minute

In [ ]:
df.columns

Index(['utc_time', 'local_time', 'df', 'd', 'ng', 'ti', 'sum_ng', 'ng_col',
       'ng_ng', 'ng_nuc', 'ng_oil', 'ng_wat', 'ng_sun', 'ng_wnd', 'ng_oth',
       'ng_unk', 'sum_trade', 'sum_imports', 'sum_exports', 'cal', 'nw', 'sw',
       'mex', 'balance_ng_d_ti', 'balance_ti_trade', 'balance_ng',
       'co2_factor_col', 'co2_factor_ng', 'co2_factor_oil',
       'co2_emissions_col', 'co2_emissions_ng', 'co2_emissions_oil',
       'co2_emissions_other', 'co2_emissions_generated',
       'co2_emissions_imported', 'co2_emissions_exported',
       'co2_emissions_consumed', 'positive_generation', 'consumed_electricity',
       'co2_emissions_intensity_for_generated_electricity',
       'co2_emissions_intensity_for_consumed_electricity', 'local_year',
       'local_month', 'local_week_of_year', 'local_day', 'local_day_of_week',
       'local_day_of_year', 'local_is_weekend', 'local_hour', 'local_minute',
       'utc_year', 'utc_month', 'utc_week_of_year', 'utc_day',
       'utc_day_of_week',

## NCAR DSWRF (Radiation) Forecast

In [ ]:
# List of Solar Resources we will analyze
solar_resource_areas = [
    ('bay_area_solar',38,-122.25),
    ('imperial_solar',33.5,-115.75),
    ('mid_central_valley_solar',38,-121),
    ('northern_central_valley_solar',39.75,-122),
    ('san_luis_obispo_solar',36,-121),
    ('south_central_valley_solar',36,-119.5),
    ('los_angeles_metro_solar',33.5,-117.5),
    ('tehachapi_solar',34.75,-117.75)
    ]

solar_resources = [x[0] for x in solar_resource_areas if '_solar' in x[0]]
solar_resources

['bay_area_solar',
 'imperial_solar',
 'mid_central_valley_solar',
 'northern_central_valley_solar',
 'san_luis_obispo_solar',
 'south_central_valley_solar',
 'los_angeles_metro_solar',
 'tehachapi_solar']

In [ ]:
# # This section extracts solar forecast data in the Data/Weather_Forecasts folder and puts them in the df_dswrf dataframe
# # It is saved to CSV so doesn't need to be run each time.
# %%time
# # This takes about 3 mins to run
# # Read the CSV file of Solar Forecasts
# df_filenames = pd.read_csv('Data/Weather_Forecasts/NCAR_RDA/solar_forecast_files_labels_shifts.csv')

# # Set up an empty df_solar
# df_dswrf = pd.DataFrame(index=pd.DatetimeIndex(['2015-01-15 00:00:00'],name='utc_time'))
# # Iterate through each row of the dataframe
# for index, row in df_filenames.iterrows():
#     # Read the data from the file in the 'filename' column
#     selected_file='Data/Weather_Forecasts/NCAR_RDA/Solar/'+row['filename']
#     selected_column_name=row['column_label']
#     selected_shift_amount=row['shift_amount']

#     # Create a dataframe for each imported file
#     df_data = pd.read_csv(selected_file)
#     df_data['utc_time']=pd.to_datetime(df_data['Date'].astype(str)+" "+df_data['Time'].astype(str))
#     df_data = df_data.set_index('utc_time').drop(['Date','Time'], axis=1)
#     df_data.columns = [selected_column_name]
#     df_data = df_data[~df_data.index.duplicated(keep='first')] # necessary to make sure there are no duplicated timestamps
#     df_data.index = df_data.index
#     print(f'\rNow importing data: {selected_column_name}', end='')
    
#     # Shift the forecast data forward in time to align it with the present power generated
#     df_data.index = df_data.index + pd.DateOffset(hours=-selected_shift_amount)  # primary offset here
#     #df_data.index = df_data.index + pd.DateOffset(hours=0)  # additional offset possible (change from 0 for further fine-tuning)

#     # Keep the dataframe defragmented with below statement
#     df_dswrf = df_dswrf.copy()

#     # Merge each imported data file into the main dataframe
#     try:
#       df_dswrf = df_dswrf.merge(df_data, how='outer', left_on='utc_time', right_on='utc_time')
#     except IndexError:
#       df_dswrf = df_data

In [ ]:
# # Write df_dswrf dataframe to csv file
# df_dswrf.to_csv('Data/Aggregated_Data/aggregated_dswrf_train.csv', sep=',', encoding='utf-8',date_format='%Y-%m-%d %H:%M:%S')

In [ ]:
# # Quadri-daily df_dswrf can be re-read from this checkpoint
# df_dswrf = pd.read_csv('Data/Aggregated_Data/aggregated_dswrf_train.csv', index_col='utc_time', parse_dates=True)
# df_dswrf.index = df_dswrf.index.tz_localize('UTC').tz_convert(timezone.utc)
# df_dswrf_columns = list(df_dswrf.columns)
# print(df_dswrf_columns)
# df_dswrf.head(5)

In [ ]:
# # This section takes df_dswrf and adds additional columns:
# # the total downward shortwave radiation flux a region is forecasted to receive
# # on day 1, 2, 3, and 4.  In other words, currently the dswrf is forecasted for
# # 6-hour intervals--the new columns group the 4x daily periods into a total forecast
# # for days 1, 2, 3, and 4, for each geography
# # ds_dswrf is written to a csv file below so this does not need to be run each time.
# %%time 
# # this cell takes approximately 8 mins to run
# for name in solar_resources:
#     for j in range(0,4):
#         df_dswrf[f'{name}_day_{j+1}_total_dswrf'] = 0  # Create a new column with initial values set to 0
#         for i in range(len(df_dswrf)):
#             hour = df_dswrf.index[i].hour  # Get the hour of the current index
#             if hour == 0:
#                 if i >= 2:
#                     df_dswrf.loc[df_dswrf.index[i], f'{name}_day_{j+1}_total_dswrf'] = df_dswrf.iloc[i][f'{name}_{24*j}to{24*j+6}'] + df_dswrf.iloc[i-1][f'{name}_{24*j}to{24*j+6}'] + df_dswrf.iloc[i-2][f'{name}_{24*j}to{24*j+6}']
#             elif hour == 6:
#                 df_dswrf.loc[df_dswrf.index[i], f'{name}_day_{j+1}_total_dswrf'] = df_dswrf.iloc[i][f'{name}_{24*j+6}to{24*j+12}'] + df_dswrf.iloc[i][f'{name}_{24*j+12}to{24*j+18}'] + df_dswrf.iloc[i][f'{name}_{24*j+18}to{24*j+24}']
#             elif hour == 12:
#                 df_dswrf.loc[df_dswrf.index[i], f'{name}_day_{j+1}_total_dswrf'] = df_dswrf.iloc[i][f'{name}_{24*j}to{24*j+6}'] + df_dswrf.iloc[i][f'{name}_{24*j+6}to{24*j+12}'] + df_dswrf.iloc[i][f'{name}_{24*j+12}to{24*j+18}']
#             elif hour == 18:
#                 if i >= 1:
#                     df_dswrf.loc[df_dswrf.index[i], f'{name}_day_{j+1}_total_dswrf'] = df_dswrf.iloc[i][f'{name}_{24*j}to{24*j+6}'] + df_dswrf.iloc[i][f'{name}_{24*j+6}to{24*j+12}'] + df_dswrf.iloc[i-1][f'{name}_{24*j}to{24*j+6}']

In [ ]:
# # Writes df_dswrf to a file as a checkpoint
# df_dswrf.to_csv('Data/Aggregated_Data/aggregated_dswrf_train_with_daily_dswrf_forecasts.csv', sep=',', encoding='utf-8',date_format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Read df_dswrf from last checkpoint
df_dswrf = pd.read_csv('Data/Aggregated_Data/aggregated_dswrf_train_with_daily_dswrf_forecasts.csv', index_col='utc_time', parse_dates=True)
df_dswrf.index = df_dswrf.index.tz_localize('UTC').tz_convert(timezone.utc)
df_dswrf

,bay_area_solar_0to6,bay_area_solar_6to12,bay_area_solar_12to18,bay_area_solar_18to24,bay_area_solar_24to30,bay_area_solar_30to36,bay_area_solar_36to42,bay_area_solar_42to48,bay_area_solar_48to54,bay_area_solar_54to60,...,south_central_valley_solar_day_3_total_dswrf,south_central_valley_solar_day_4_total_dswrf,los_angeles_metro_solar_day_1_total_dswrf,los_angeles_metro_solar_day_2_total_dswrf,los_angeles_metro_solar_day_3_total_dswrf,los_angeles_metro_solar_day_4_total_dswrf,tehachapi_solar_day_1_total_dswrf,tehachapi_solar_day_2_total_dswrf,tehachapi_solar_day_3_total_dswrf,tehachapi_solar_day_4_total_dswrf
utc_time,,,,,,,,,,,,,,,,,,,,,
2015-01-15 00:00:00+00:00,10.0,0.0,50.0,307.0,10.0,0.0,60.0,319.0,10.0,0.0,...,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.0
2015-01-15 06:00:00+00:00,0.0,70.0,354.0,10.0,0.0,70.0,321.0,10.0,0.0,60.0,...,549.000,552.0,644.0,645.0,622.000,610.000,625.0,604.0,616.000,591.0
2015-01-15 12:00:00+00:00,70.0,331.0,10.0,0.0,70.0,385.0,10.0,0.0,60.0,407.0,...,541.000,571.0,644.0,648.0,640.000,560.000,624.0,607.0,619.000,598.0
2015-01-15 18:00:00+00:00,342.0,10.0,0.0,70.0,407.0,10.0,0.0,60.0,400.0,10.0,...,537.000,553.0,643.0,631.0,639.000,589.000,624.0,584.0,620.000,609.0
2015-01-16 00:00:00+00:00,10.0,0.0,70.0,377.0,10.0,0.0,60.0,375.0,10.0,0.0,...,537.000,553.0,643.0,631.0,639.000,589.000,624.0,584.0,620.000,609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-26 18:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,885.0,149.0,...,1704.912,NaN,NaN,NaN,1862.112,NaN,NaN,NaN,1901.072,NaN
2023-02-14 18:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1397.016,1421.224,NaN,NaN,NaN,NaN
2023-02-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1397.192,1416.264,NaN,NaN,NaN,NaN


In [ ]:
# We are going to only use some of the columns, so we are trimming the columns here
df_dswrf_trimmed = df_dswrf[
 ['bay_area_solar_day_1_total_dswrf',
 'bay_area_solar_day_2_total_dswrf',
 'bay_area_solar_day_3_total_dswrf',
 'bay_area_solar_day_4_total_dswrf',
 'imperial_solar_day_1_total_dswrf',
 'imperial_solar_day_2_total_dswrf',
 'imperial_solar_day_3_total_dswrf',
 'imperial_solar_day_4_total_dswrf',
 'mid_central_valley_solar_day_1_total_dswrf',
 'mid_central_valley_solar_day_2_total_dswrf',
 'mid_central_valley_solar_day_3_total_dswrf',
 'mid_central_valley_solar_day_4_total_dswrf',
 'northern_central_valley_solar_day_1_total_dswrf',
 'northern_central_valley_solar_day_2_total_dswrf',
 'northern_central_valley_solar_day_3_total_dswrf',
 'northern_central_valley_solar_day_4_total_dswrf',
 'san_luis_obispo_solar_day_1_total_dswrf',
 'san_luis_obispo_solar_day_2_total_dswrf',
 'san_luis_obispo_solar_day_3_total_dswrf',
 'san_luis_obispo_solar_day_4_total_dswrf',
 'south_central_valley_solar_day_1_total_dswrf',
 'south_central_valley_solar_day_2_total_dswrf',
 'south_central_valley_solar_day_3_total_dswrf',
 'south_central_valley_solar_day_4_total_dswrf',
 'los_angeles_metro_solar_day_1_total_dswrf',
 'los_angeles_metro_solar_day_2_total_dswrf',
 'los_angeles_metro_solar_day_3_total_dswrf',
 'los_angeles_metro_solar_day_4_total_dswrf',
 'tehachapi_solar_day_1_total_dswrf',
 'tehachapi_solar_day_2_total_dswrf',
 'tehachapi_solar_day_3_total_dswrf',
 'tehachapi_solar_day_4_total_dswrf']]

In [ ]:
# Factor in Solar Resource Weights - this comes from CEC data on capacity of each region
# solar_resource_weights = [
#     ('bay_area', 168.75, 0.0116833268),
#     ('south_central_valley', 3851.83, 0.0116833268),
#     ('mid_central_valley', 772.76, 0.05350167478),
#     ('northern_central_valley', 41.6, 0.002880156415),
#     ('los_angeles_metro',	276.01, 0.0191094224),
#     ('tehachapi', 4789.95, 0.3316299331),
#     ('san_luis_obispo',	318.08, 0.02202211905),
#     ('imperial', 4224.68, 0.2924937308)
# ]
total_solar_resource_weights = 14443.66
solar_resource_weights = {
    'bay_area': 0.0116833268,
    'south_central_valley': 0.0116833268,
    'mid_central_valley': 0.05350167478,
    'northern_central_valley': 0.002880156415,
    'los_angeles_metro': 0.0191094224,
    'tehachapi': 0.3316299331,
    'san_luis_obispo': 0.02202211905,
    'imperial': 0.2924937308
}
solar_resource_weights = {key+'_solar': float(value) for key, value in solar_resource_weights.items()} # convert values to floats and tweak key

In [ ]:
df_main = df # Renaming for convenience

In [ ]:
df_main

,utc_time,local_time,df,d,ng,ti,sum_ng,ng_col,ng_ng,ng_nuc,...,local_minute,utc_year,utc_month,utc_week_of_year,utc_day,utc_day_of_week,utc_day_of_year,utc_is_weekend,utc_hour,utc_minute
utc_time_with_offset,,,,,,,,,,,,,,,,,,,,,
2015-07-01 08:00:00+00:00,2015-07-01 08:00:00,2015-07-01 01:00:00,35264,38210.0,31005.0,-7412.0,NaN,NaN,NaN,NaN,...,0,2015,7,27,1,2,182,False,8,0
2015-07-01 09:00:00+00:00,2015-07-01 09:00:00,2015-07-01 02:00:00,32894,35171.0,28760.0,-6671.0,NaN,NaN,NaN,NaN,...,0,2015,7,27,1,2,182,False,9,0
2015-07-01 10:00:00+00:00,2015-07-01 10:00:00,2015-07-01 03:00:00,31360,33243.0,27179.0,-6306.0,NaN,NaN,NaN,NaN,...,0,2015,7,27,1,2,182,False,10,0
2015-07-01 11:00:00+00:00,2015-07-01 11:00:00,2015-07-01 04:00:00,30579,31955.0,25660.0,-6551.0,NaN,NaN,NaN,NaN,...,0,2015,7,27,1,2,182,False,11,0
2015-07-01 12:00:00+00:00,2015-07-01 12:00:00,2015-07-01 05:00:00,30723,31199.0,24961.0,-6491.0,NaN,NaN,NaN,NaN,...,0,2015,7,27,1,2,182,False,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-24 03:00:00+00:00,2023-03-24 03:00:00,2023-03-23 20:00:00,32378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,2023,3,12,24,4,83,False,3,0
2023-03-24 04:00:00+00:00,2023-03-24 04:00:00,2023-03-23 21:00:00,32208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,2023,3,12,24,4,83,False,4,0
2023-03-24 05:00:00+00:00,2023-03-24 05:00:00,2023-03-23 22:00:00,31200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,2023,3,12,24,4,83,False,5,0


In [ ]:
# Merge df_dswrf to df_main
df_main = df_main.merge(df_dswrf_trimmed, how='left', left_index=True, right_index=True)

# Drop rows after 2/15/23 12:00:00 because no DSWRF data beyond that point
df_main = df_main[df_main.index <= '2023-02-15 12:00:00+00:00']
df_main = df_main.ffill()
df_main

,utc_time,local_time,df,d,ng,ti,sum_ng,ng_col,ng_ng,ng_nuc,...,south_central_valley_solar_day_3_total_dswrf,south_central_valley_solar_day_4_total_dswrf,los_angeles_metro_solar_day_1_total_dswrf,los_angeles_metro_solar_day_2_total_dswrf,los_angeles_metro_solar_day_3_total_dswrf,los_angeles_metro_solar_day_4_total_dswrf,tehachapi_solar_day_1_total_dswrf,tehachapi_solar_day_2_total_dswrf,tehachapi_solar_day_3_total_dswrf,tehachapi_solar_day_4_total_dswrf
utc_time_with_offset,,,,,,,,,,,,,,,,,,,,,
2015-07-01 08:00:00+00:00,2015-07-01 08:00:00,2015-07-01 01:00:00,35264,38210.0,31005.0,-7412.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01 09:00:00+00:00,2015-07-01 09:00:00,2015-07-01 02:00:00,32894,35171.0,28760.0,-6671.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01 10:00:00+00:00,2015-07-01 10:00:00,2015-07-01 03:00:00,31360,33243.0,27179.0,-6306.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01 11:00:00+00:00,2015-07-01 11:00:00,2015-07-01 04:00:00,30579,31955.0,25660.0,-6551.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01 12:00:00+00:00,2015-07-01 12:00:00,2015-07-01 05:00:00,30723,31199.0,24961.0,-6491.0,NaN,NaN,NaN,NaN,...,1342.000,1399.000,1133.000,1386.000,1128.000,1396.000,1136.000,1406.000,1248.000,1417.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-15 08:00:00+00:00,2023-02-15 08:00:00,2023-02-15 00:00:00,27885,30372.0,23650.0,-4279.0,23681.0,349.0,13737.0,2270.0,...,618.016,512.272,161.532,868.688,626.592,820.672,815.328,853.776,709.712,604.976
2023-02-15 09:00:00+00:00,2023-02-15 09:00:00,2023-02-15 01:00:00,26621,28619.0,23058.0,-3772.0,23089.0,318.0,13679.0,2267.0,...,618.016,512.272,161.532,868.688,626.592,820.672,815.328,853.776,709.712,604.976
2023-02-15 10:00:00+00:00,2023-02-15 10:00:00,2023-02-15 02:00:00,25768,27772.0,22475.0,-3690.0,22506.0,254.0,13687.0,2268.0,...,618.016,512.272,161.532,868.688,626.592,820.672,815.328,853.776,709.712,604.976


## Solar Radiation (from Space) Model

In [ ]:
# These are helper functions you need for calculations later on

# Define a function to get the sunrise and sunset times for a given date and location
def get_solar_noon(date, lat, lng):
    url = f"https://api.sunrise-sunset.org/json?lat={lat}&lng={lng}&date={date}&formatted=0"
    response = requests.get(url)
    data = response.json()["results"]
    solar_noon = data["solar_noon"]
    day_length = data["day_length"]
    return solar_noon, day_length

# Calculate the value of the solar radiation curve when it's daytime
def solar_radiation_curve(time_diff, day_length):
    """
    Calculates the solar radiation curve at a given time of day.

    Arguments:
    time_diff -- difference between current time and solar noon in seconds
    day_length -- length of day in seconds

    Returns:
    The intensity of solar radiation in W/m^2 at the given time of day.
    """    
    # if night
    if abs(time_diff) > (day_length/2):
        intensity = 0

    # if daytime
    else:
        I_0 = 1367  # solar constant in W/m^2
        radians = (math.pi * time_diff) / day_length # calculate radians for sine function
        intensity = I_0 * math.cos(radians) # calculate intensity of solar radiation

    return intensity

In [ ]:
# # This function uses the get_solar_noon function to get solar_noon and daylength
# # at a given latitude and longitude from the sunrise-sunset.org API and writes
# # writes the to csv files.  It only needs to be run once, runtime is approximately 1.5 hours

# %%time
# for location in solar_resource_areas:
#     name, lat, lng = location
#     file_name = f"Data/Solar_Radiation/{name}_radiation.csv"
#     with open(file_name, "w") as csv_file:
#         writer = csv.writer(csv_file)
#         writer.writerow(["date", "solar_noon", "day_length"])
#         # Loop through every day since 1/1/2017
#         for year in range(2017, 2023):
#             for month in range(1, 13):
#                 for day in range(1, 32):
#                     # Ignore invalid dates (e.g. Feb 30)
#                     try:
#                         date = f"{year:04}-{month:02}-{day:02}"
#                         solar_noon, day_length = get_solar_noon(date, lat, lng)
#                         writer.writerow([date, solar_noon, day_length])
#                         print(f'\rNow writing data for: {location} {date} {solar_noon} {day_length}', end='')
#                     except KeyError:
#                         pass
#                     except TypeError:
#                         pass

In [ ]:
%%time
# This section pulls in the solar_noon and day_length data from the files generated
# in the previous section, adds them to the df_main dataframe.  With that information,
# we can use the datetime to figure out exactly what the solar irradiation wave
# pulse can be at a given timestamp.  The solar irradiation curve for a given geography
# is added as a column.  Additionally, for each geography, there are additional columns
# the forecast the solar radiation curve for days 1, 2, 3, and 4, based on the forecasted
# dswrf for that day.

df_main['total_weighted_radiation_now']=0
df_main['total_weighted_radiation_1_day_ahead']=0
df_main['total_weighted_radiation_2_days_ahead']=0
df_main['total_weighted_radiation_3_days_ahead']=0
df_main['total_weighted_radiation_4_days_ahead']=0

for solar_resource in solar_resources:
    # Create dataframe for joining to df_main
    # Read solar radiation file for each solar resource area
    df_solar_noon = pd.read_csv(f'Data/Solar_Radiation/{solar_resource}_radiation.csv')

    # Convert dates to datetimes and prefix solar_noon and day_length with solar_resouces
    df_solar_noon['date'] = pd.to_datetime(df_solar_noon['date'], utc=True)
    df_solar_noon[f'{solar_resource}_solar_noon'] = pd.to_datetime(df_solar_noon['solar_noon'])
    df_solar_noon[f'{solar_resource}_day_length'] = df_solar_noon['day_length']

    # Create a new column 'midnight' with the datetime for the previous UTC midnight
    df_solar_noon['midnight'] = df_solar_noon[f'{solar_resource}_solar_noon'].apply(lambda x: x.replace(hour=8, minute=0, second=0, microsecond=0))
    df_solar_noon = df_solar_noon.drop(['date', 'solar_noon', 'day_length'], axis=1)

    # Set midnight as index
    df_solar_noon.set_index('midnight', inplace=True)

    # Merge newly created df_solar_noon into df_main
    df_main = pd.merge_asof(df_main, df_solar_noon, left_index=True, right_index=True, direction='backward')
    df_main[f'{solar_resource}_time_diff'] = (df_main.index-df_main[f'{solar_resource}_solar_noon']).dt.total_seconds()
    df_main[f'{solar_resource}_radiation_now'] = df_main.apply(lambda row: solar_radiation_curve(row[f'{solar_resource}_time_diff'], row[f'{solar_resource}_day_length']), axis=1)
    df_main = df_main.drop([f'{solar_resource}_time_diff', f'{solar_resource}_day_length', f'{solar_resource}_solar_noon'], axis=1)

    # Create shifted versions for forecasts
    df_main[f'{solar_resource}_weighted_radiation_1_day_ahead'] = df_main[f'{solar_resource}_radiation_now'].shift(-24) * df_main[f'{solar_resource}_day_1_total_dswrf'] / 1367
    df_main[f'{solar_resource}_weighted_radiation_2_days_ahead'] = df_main[f'{solar_resource}_radiation_now'].shift(-48) * df_main[f'{solar_resource}_day_2_total_dswrf'] / 1367
    df_main[f'{solar_resource}_weighted_radiation_3_days_ahead'] = df_main[f'{solar_resource}_radiation_now'].shift(-72) * df_main[f'{solar_resource}_day_3_total_dswrf'] / 1367
    df_main[f'{solar_resource}_weighted_radiation_4_days_ahead'] = df_main[f'{solar_resource}_radiation_now'].shift(-96) * df_main[f'{solar_resource}_day_4_total_dswrf'] / 1367

    # Get weighted forecasts based on capacity weightings of region
    df_main['total_weighted_radiation_now'] = df_main['total_weighted_radiation_now'] + df_main[f'{solar_resource}_radiation_now'].multiply(solar_resource_weights[solar_resource])
    df_main['total_weighted_radiation_1_day_ahead'] = df_main['total_weighted_radiation_1_day_ahead']+df_main[f'{solar_resource}_weighted_radiation_1_day_ahead'].multiply(solar_resource_weights[solar_resource])
    df_main['total_weighted_radiation_2_days_ahead'] = df_main['total_weighted_radiation_2_days_ahead']+df_main[f'{solar_resource}_weighted_radiation_2_days_ahead'].multiply(solar_resource_weights[solar_resource])
    df_main['total_weighted_radiation_3_days_ahead'] = df_main['total_weighted_radiation_3_days_ahead']+df_main[f'{solar_resource}_weighted_radiation_3_days_ahead'].multiply(solar_resource_weights[solar_resource])
    df_main['total_weighted_radiation_4_days_ahead'] = df_main['total_weighted_radiation_4_days_ahead']+df_main[f'{solar_resource}_weighted_radiation_4_days_ahead'].multiply(solar_resource_weights[solar_resource])

CPU times: user 18.9 s, sys: 2.58 s, total: 21.5 s
Wall time: 26.7 s


In [ ]:
# Clarify if daytime is occurring if radiation > 0
df_main['is_daytime'] = df_main['total_weighted_radiation_now'] > 0

# Drop all radiation columns except the totals
df_main = df_main.drop(columns=['bay_area_solar_day_1_total_dswrf',
 'bay_area_solar_day_2_total_dswrf',
 'bay_area_solar_day_3_total_dswrf',
 'bay_area_solar_day_4_total_dswrf',
 'imperial_solar_day_1_total_dswrf',
 'imperial_solar_day_2_total_dswrf',
 'imperial_solar_day_3_total_dswrf',
 'imperial_solar_day_4_total_dswrf',
 'mid_central_valley_solar_day_1_total_dswrf',
 'mid_central_valley_solar_day_2_total_dswrf',
 'mid_central_valley_solar_day_3_total_dswrf',
 'mid_central_valley_solar_day_4_total_dswrf',
 'northern_central_valley_solar_day_1_total_dswrf',
 'northern_central_valley_solar_day_2_total_dswrf',
 'northern_central_valley_solar_day_3_total_dswrf',
 'northern_central_valley_solar_day_4_total_dswrf',
 'san_luis_obispo_solar_day_1_total_dswrf',
 'san_luis_obispo_solar_day_2_total_dswrf',
 'san_luis_obispo_solar_day_3_total_dswrf',
 'san_luis_obispo_solar_day_4_total_dswrf',
 'south_central_valley_solar_day_1_total_dswrf',
 'south_central_valley_solar_day_2_total_dswrf',
 'south_central_valley_solar_day_3_total_dswrf',
 'south_central_valley_solar_day_4_total_dswrf',
 'los_angeles_metro_solar_day_1_total_dswrf',
 'los_angeles_metro_solar_day_2_total_dswrf',
 'los_angeles_metro_solar_day_3_total_dswrf',
 'los_angeles_metro_solar_day_4_total_dswrf',
 'tehachapi_solar_day_1_total_dswrf',
 'tehachapi_solar_day_2_total_dswrf',
 'tehachapi_solar_day_3_total_dswrf',
 'tehachapi_solar_day_4_total_dswrf',
 'bay_area_solar_radiation_now',
 'bay_area_solar_weighted_radiation_1_day_ahead',
 'bay_area_solar_weighted_radiation_2_days_ahead',
 'bay_area_solar_weighted_radiation_3_days_ahead',
 'bay_area_solar_weighted_radiation_4_days_ahead',
 'imperial_solar_radiation_now',
 'imperial_solar_weighted_radiation_1_day_ahead',
 'imperial_solar_weighted_radiation_2_days_ahead',
 'imperial_solar_weighted_radiation_3_days_ahead',
 'imperial_solar_weighted_radiation_4_days_ahead',
 'mid_central_valley_solar_radiation_now',
 'mid_central_valley_solar_weighted_radiation_1_day_ahead',
 'mid_central_valley_solar_weighted_radiation_2_days_ahead',
 'mid_central_valley_solar_weighted_radiation_3_days_ahead',
 'mid_central_valley_solar_weighted_radiation_4_days_ahead',
 'northern_central_valley_solar_radiation_now',
 'northern_central_valley_solar_weighted_radiation_1_day_ahead',
 'northern_central_valley_solar_weighted_radiation_2_days_ahead',
 'northern_central_valley_solar_weighted_radiation_3_days_ahead',
 'northern_central_valley_solar_weighted_radiation_4_days_ahead',
 'san_luis_obispo_solar_radiation_now',
 'san_luis_obispo_solar_weighted_radiation_1_day_ahead',
 'san_luis_obispo_solar_weighted_radiation_2_days_ahead',
 'san_luis_obispo_solar_weighted_radiation_3_days_ahead',
 'san_luis_obispo_solar_weighted_radiation_4_days_ahead',
 'south_central_valley_solar_radiation_now',
 'south_central_valley_solar_weighted_radiation_1_day_ahead',
 'south_central_valley_solar_weighted_radiation_2_days_ahead',
 'south_central_valley_solar_weighted_radiation_3_days_ahead',
 'south_central_valley_solar_weighted_radiation_4_days_ahead',
 'los_angeles_metro_solar_radiation_now',
 'los_angeles_metro_solar_weighted_radiation_1_day_ahead',
 'los_angeles_metro_solar_weighted_radiation_2_days_ahead',
 'los_angeles_metro_solar_weighted_radiation_3_days_ahead',
 'los_angeles_metro_solar_weighted_radiation_4_days_ahead',
 'tehachapi_solar_radiation_now',
 'tehachapi_solar_weighted_radiation_1_day_ahead',
 'tehachapi_solar_weighted_radiation_2_days_ahead',
 'tehachapi_solar_weighted_radiation_3_days_ahead',
 'tehachapi_solar_weighted_radiation_4_days_ahead'], axis=1)

In [ ]:
# # NOT CURRENTLY USED -- Reload df_main dataframe from previously saved checkpoint
# df_train = pd.read_csv('Data/Aggregated_Data/train_caiso_eia_and_radiation.csv', index_col='utc_time', parse_dates=True)
# df_test = pd.read_csv('Data/Aggregated_Data/test_caiso_eia_and_radiation.csv', index_col='utc_time', parse_dates=True)
# df_main = pd.concat([df_train, df_test], axis=0)

## Split into Train and Test Sets

In [ ]:
# Clean dataframe by rearranging columns
df_main = df_main.reindex(columns=[
    # These are the time metrics I think we should keep
    'utc_time', 'local_time', 'local_year', 'local_month', 'local_week_of_year', 'local_day',
    'local_day_of_week', 'local_day_of_year', 'local_is_weekend', 'utc_hour', 'local_hour', 'local_minute',
    'is_daytime',
    # These are the ones I think are redundant
    'utc_year', 'utc_month', 'utc_week_of_year', 'utc_day', 'utc_day_of_week', 'utc_day_of_year',
    'utc_is_weekend', 'utc_minute',
    # These are from EIA
    'df', 'd', 'ng', 'ti', 'sum_ng', 'ng_col',
    'ng_ng', 'ng_nuc', 'ng_oil', 'ng_wat', 'ng_sun', 'ng_wnd', 'ng_oth',
    'ng_unk', 'sum_trade', 'sum_imports', 'sum_exports', 'cal', 'nw', 'sw',
    'mex', 'balance_ng_d_ti', 'balance_ti_trade', 'balance_ng',
    'co2_factor_col', 'co2_factor_ng', 'co2_factor_oil',
    'co2_emissions_col', 'co2_emissions_ng', 'co2_emissions_oil',
    'co2_emissions_other', 'co2_emissions_generated',
    'co2_emissions_imported', 'co2_emissions_exported',
    'co2_emissions_consumed', 'positive_generation', 'consumed_electricity',
    'co2_emissions_intensity_for_generated_electricity',
    'co2_emissions_intensity_for_consumed_electricity',
    # These are solar radiation related
    'total_weighted_radiation_now',
    'total_weighted_radiation_1_day_ahead',
    'total_weighted_radiation_2_days_ahead',
    'total_weighted_radiation_3_days_ahead',
    'total_weighted_radiation_4_days_ahead'])

In [ ]:
# # Split into training and test sets
df_main_train = df_main[df_main['local_year'] < 2022]
df_main_test = df_main[df_main['local_year'] >= 2022]

In [ ]:
# Write to csv file
df_main_train.to_csv('Data/Aggregated_Data/train_eia_radiation_clean.csv', index=True)
df_main_test.to_csv('Data/Aggregated_Data/test_eia_radiation_clean.csv', index=True)

# Load Pipeline Data From File

In [ ]:
df_train = pd.read_csv('Data/Aggregated_Data/train_eia_radiation_clean.csv', index_col='utc_time', parse_dates=True)

In [ ]:
df_train

,utc_time_with_offset,local_time,local_year,local_month,local_week_of_year,local_day,local_day_of_week,local_day_of_year,local_is_weekend,utc_hour,...,co2_emissions_consumed,positive_generation,consumed_electricity,co2_emissions_intensity_for_generated_electricity,co2_emissions_intensity_for_consumed_electricity,total_weighted_radiation_now,total_weighted_radiation_1_day_ahead,total_weighted_radiation_2_days_ahead,total_weighted_radiation_3_days_ahead,total_weighted_radiation_4_days_ahead
utc_time,,,,,,,,,,,,,,,,,,,,,
2015-07-01 08:00:00,2015-07-01 08:00:00+00:00,2015-07-01 01:00:00,2015,7,27,1,2,182,False,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01 09:00:00,2015-07-01 09:00:00+00:00,2015-07-01 02:00:00,2015,7,27,1,2,182,False,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01 10:00:00,2015-07-01 10:00:00+00:00,2015-07-01 03:00:00,2015,7,27,1,2,182,False,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01 11:00:00,2015-07-01 11:00:00+00:00,2015-07-01 04:00:00,2015,7,27,1,2,182,False,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01 12:00:00,2015-07-01 12:00:00+00:00,2015-07-01 05:00:00,2015,7,27,1,2,182,False,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 03:00:00,2022-01-01 03:00:00+00:00,2021-12-31 19:00:00,2021,12,52,31,4,365,False,3,...,7773.260494,22780.0,31870.0,0.508546,0.537718,0.0,0.0,0.0,0.0,0.0
2022-01-01 04:00:00,2022-01-01 04:00:00+00:00,2021-12-31 20:00:00,2021,12,52,31,4,365,False,4,...,7687.742276,22045.0,30984.0,0.520778,0.547010,0.0,0.0,0.0,0.0,0.0
2022-01-01 05:00:00,2022-01-01 05:00:00+00:00,2021-12-31 21:00:00,2021,12,52,31,4,365,False,5,...,7721.388254,21060.0,30381.0,0.534327,0.560308,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_train[['local_time', 'total_weighted_radiation_now', 'is_daytime']].tail(30)

,local_time,total_weighted_radiation_now,is_daytime
utc_time,,,
2021-12-31 02:00:00,2021-12-30 18:00:00,0.000000,False
2021-12-31 03:00:00,2021-12-30 19:00:00,0.000000,False
2021-12-31 04:00:00,2021-12-30 20:00:00,0.000000,False
2021-12-31 05:00:00,2021-12-30 21:00:00,0.000000,False
2021-12-31 06:00:00,2021-12-30 22:00:00,0.000000,False
2021-12-31 07:00:00,2021-12-30 23:00:00,0.000000,False
2021-12-31 08:00:00,2021-12-31 00:00:00,0.000000,False
2021-12-31 09:00:00,2021-12-31 01:00:00,0.000000,False
2021-12-31 10:00:00,2021-12-31 02:00:00,0.000000,False


In [ ]:
list(df_train.columns)

['utc_time_with_offset',
 'local_time',
 'local_year',
 'local_month',
 'local_week_of_year',
 'local_day',
 'local_day_of_week',
 'local_day_of_year',
 'local_is_weekend',
 'utc_hour',
 'local_hour',
 'local_minute',
 'is_daytime',
 'utc_year',
 'utc_month',
 'utc_week_of_year',
 'utc_day',
 'utc_day_of_week',
 'utc_day_of_year',
 'utc_is_weekend',
 'utc_minute',
 'df',
 'd',
 'ng',
 'ti',
 'sum_ng',
 'ng_col',
 'ng_ng',
 'ng_nuc',
 'ng_oil',
 'ng_wat',
 'ng_sun',
 'ng_wnd',
 'ng_oth',
 'ng_unk',
 'sum_trade',
 'sum_imports',
 'sum_exports',
 'cal',
 'nw',
 'sw',
 'mex',
 'balance_ng_d_ti',
 'balance_ti_trade',
 'balance_ng',
 'co2_factor_col',
 'co2_factor_ng',
 'co2_factor_oil',
 'co2_emissions_col',
 'co2_emissions_ng',
 'co2_emissions_oil',
 'co2_emissions_other',
 'co2_emissions_generated',
 'co2_emissions_imported',
 'co2_emissions_exported',
 'co2_emissions_consumed',
 'positive_generation',
 'consumed_electricity',
 'co2_emissions_intensity_for_generated_electricity',
 'co2_em